In [3]:
import pandas as pd

ruta = '../../data/processed/actividades_infantiles_madrid_19-04-2025.csv'
df = pd.read_csv(ruta)

In [5]:
df.head()

,titulo,tipo,lugar,distrito,fecha_inicio,fecha_fin,url
0,Talleres creativos en idioma serbio,CURSOS Y TALLERES,Biblioteca Pública Municipal Iván de Vargas (C...,CENTRO,domingo 22 de septiembre de 2024,domingo 22 de junio de 2025,https://www.madrid.es/portales/munimadrid/es/I...
1,Tertulias en Inglés (infantil) Biblioteca Vall...,CURSOS Y TALLERES,Biblioteca Pública Municipal Vallecas (Puente ...,PUENTE DE VALLECAS,miércoles 25 de septiembre de 2024,miércoles 11 de junio de 2025,https://www.madrid.es/portales/munimadrid/es/I...
2,Club de lectura (infantil) Biblioteca David Gi...,"CLUBES DE LECTURA, \r\n \r\n ...",Biblioteca Pública Municipal David Gistau (Sal...,SALAMANCA,lunes 30 de septiembre de 2024,lunes 9 de junio de 2025,https://www.madrid.es/portales/munimadrid/es/I...
3,Taller de cómic (Infantil) Biblioteca Pozo del...,CURSOS Y TALLERES,Biblioteca Pública Municipal Pozo del Tío Raim...,PUENTE DE VALLECAS,lunes 30 de septiembre de 2024,lunes 2 de junio de 2025,https://www.madrid.es/portales/munimadrid/es/I...
4,Club de lectura (infantil) Biblioteca San Fermín,"CLUBES DE LECTURA, \r\n \r\n ...",Biblioteca Pública Municipal San Fermín (Usera),USERA,miércoles 2 de octubre de 2024,miércoles 18 de junio de 2025,https://www.madrid.es/portales/munimadrid/es/I...


In [6]:
import sqlite3
import json

# Limpiar y preparar los datos
def clean_data(df):
    # Crear una copia para no modificar el original
    df_clean = df.copy()
    
    # Rellenar valores nulos
    df_clean = df_clean.fillna({
        'titulo': 'Sin título',
        'tipo': 'No especificado',
        'lugar': 'No especificado',
        'distrito': 'No especificado',
        'fecha_inicio': 'No especificado',
        'fecha_fin': 'No especificado',
        'url': ''
    })
    
    # Normalizar nombres de columnas (quitar espacios, minúsculas)
    df_clean.columns = [col.lower().replace(' ', '_') for col in df_clean.columns]
    
    return df_clean

df_clean = clean_data(df)


In [7]:
# 1. Crear una base de datos SQLite
def create_sqlite_db():
    conn = sqlite3.connect('actividades_infantiles.db')
    df_clean.to_sql('actividades', conn, if_exists='replace', index=False)
    conn.close()
    return "Base de datos SQLite creada: actividades_infantiles.db"

In [9]:
# 2. Exportar a formato SQL (para MySQL/PostgreSQL)
def create_sql_script():
    # Crear script SQL para MySQL
    with open('actividades_mysql.sql', 'w', encoding='utf-8') as f:
        # Crear tabla
        f.write("CREATE TABLE IF NOT EXISTS actividades (\n")
        f.write("    id INT AUTO_INCREMENT PRIMARY KEY,\n")
        f.write("    titulo VARCHAR(255),\n")
        f.write("    tipo VARCHAR(100),\n")
        f.write("    lugar VARCHAR(255),\n")
        f.write("    distrito VARCHAR(100),\n")
        f.write("    fecha_inicio VARCHAR(100),\n")
        f.write("    fecha_fin VARCHAR(100),\n")
        f.write("    url TEXT\n")
        f.write(");\n\n")
        
        # Insertar datos
        for _, row in df_clean.iterrows():
            values = []
            for val in row:
                if pd.isna(val):
                    values.append("NULL")
                else:
                    # Escapar comillas simples para SQL
                    val_str = str(val).replace("'", "''")
                    values.append(f"'{val_str}'")
            
            insert_sql = "INSERT INTO actividades (titulo, tipo, lugar, distrito, fecha_inicio, fecha_fin, url) VALUES ({});".format(", ".join(values))
            f.write(insert_sql + "\n")
    
    return "Script SQL creado: actividades_mysql.sql"

In [10]:
# 3. Exportar a JSON para bases de datos NoSQL
def create_json():
    # Convertir a formato JSON
    json_data = df_clean.to_json(orient='records', force_ascii=False, indent=4)
    
    with open('actividades.json', 'w', encoding='utf-8') as f:
        f.write(json_data)
    
    return "Archivo JSON creado: actividades.json"

In [11]:
# 4. Crear un script Python para MongoDB
def create_mongodb_script():
    with open('importar_mongodb.py', 'w', encoding='utf-8') as f:
        f.write("from pymongo import MongoClient\n")
        f.write("import json\n\n")
        f.write("# Conectar a MongoDB (ajusta la URL según tu configuración)\n")
        f.write("client = MongoClient('mongodb://localhost:27017/')\n")
        f.write("db = client['actividades_infantiles']\n")
        f.write("collection = db['actividades']\n\n")
        f.write("# Importar datos desde el archivo JSON\n")
        f.write("with open('actividades.json', 'r', encoding='utf-8') as file:\n")
        f.write("    actividades = json.load(file)\n\n")
        f.write("# Insertar datos en la colección\n")
        f.write("collection.insert_many(actividades)\n\n")
        f.write("print('Se importaron {} actividades a MongoDB'.format(len(actividades)))\n")
    
    return "Script Python para MongoDB creado: importar_mongodb.py"

In [12]:
# Ejecutar todas las funciones
sqlite_result = create_sqlite_db()
sql_result = create_sql_script()
json_result = create_json()
mongodb_result = create_mongodb_script()

In [16]:
import sqlite3
import psycopg2
import os

# Primero, vamos a examinar la estructura de la base de datos SQLite
def explore_sqlite_db():
    # Conectar a la base de datos SQLite
    conn = sqlite3.connect('actividades_infantiles.db')
    
    # Obtener lista de tablas
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    
    print("Tablas en la base de datos SQLite:")
    for table in tables:
        print(f"- {table[0]}")
    
    # Para cada tabla, mostrar su estructura y algunos datos
    for table in tables:
        table_name = table[0]
        print(f"\nEstructura de la tabla '{table_name}':")
        
        # Obtener información de columnas
        cursor.execute(f"PRAGMA table_info({table_name});")
        columns = cursor.fetchall()
        for col in columns:
            print(f"  {col[1]} ({col[2]})")
        
        # Mostrar algunos datos
        cursor.execute(f"SELECT * FROM {table_name} LIMIT 5;")
        rows = cursor.fetchall()
        
        # Crear un DataFrame para mostrar los datos de manera más legible
        df = pd.DataFrame(rows, columns=[col[1] for col in columns])
        print("\nMuestra de datos:")
        print(df)
    
    # Cerrar la conexión
    conn.close()
    
    return tables

# Función para generar script SQL para PostgreSQL
def generate_postgres_script(tables):
    conn = sqlite3.connect('actividades_infantiles.db')
    
    # Crear un script SQL para PostgreSQL
    with open('migrar_a_postgres.sql', 'w', encoding='utf-8') as f:
        for table in tables:
            table_name = table[0]
            
            # Obtener información de columnas
            cursor = conn.cursor()
            cursor.execute(f"PRAGMA table_info({table_name});")
            columns = cursor.fetchall()
            
            # Crear sentencia CREATE TABLE
            f.write(f"-- Tabla: {table_name}\n")
            f.write(f"DROP TABLE IF EXISTS {table_name};\n")
            f.write(f"CREATE TABLE {table_name} (\n")
            
            # Mapear tipos de SQLite a PostgreSQL
            column_defs = []
            for col in columns:
                col_name = col[1]
                col_type = col[2].upper()
                
                # Mapeo de tipos
                if col_type == 'INTEGER' and col[5] == 1:  # Es clave primaria
                    pg_type = 'SERIAL PRIMARY KEY'
                elif col_type == 'INTEGER':
                    pg_type = 'INTEGER'
                elif col_type == 'REAL':
                    pg_type = 'REAL'
                elif col_type.startswith('VARCHAR') or col_type == 'TEXT':
                    pg_type = 'TEXT'
                else:
                    pg_type = 'TEXT'  # Por defecto
                
                column_defs.append(f"    {col_name} {pg_type}")
            
            f.write(",\n".join(column_defs))
            f.write("\n);\n\n")
            
            # Obtener datos para INSERT
            cursor.execute(f"SELECT * FROM {table_name};")
            rows = cursor.fetchall()
            
            # Generar sentencias INSERT
            if rows:
                f.write(f"-- Datos para {table_name}\n")
                for row in rows:
                    values = []
                    for val in row:
                        if val is None:
                            values.append('NULL')
                        elif isinstance(val, str):
                            # Escapar comillas simples
                            escaped_val = val.replace("'", "''")
                            values.append(f"'{escaped_val}'")
                        else:
                            values.append(str(val))
                    
                    col_names = [col[1] for col in columns]
                    f.write(f"INSERT INTO {table_name} ({', '.join(col_names)}) VALUES ({', '.join(values)});\n")
                
                f.write("\n")
    
    conn.close()
    
    print("\nScript SQL para PostgreSQL generado: migrar_a_postgres.sql")
    print("Puedes usar este script para importar los datos a PostgreSQL.")

In [20]:
# Crear un script de migración más simple y robusto
with open('migrar_simple.py', 'w', encoding='utf-8') as f:
    f.write("""
import sqlite3
import psycopg2
import pandas as pd
import sys

def migrate_sqlite_to_postgres():
    try:
        # Conectar a SQLite
        print("Conectando a SQLite...")
        sqlite_conn = sqlite3.connect('actividades_infantiles.db')
        
        # Leer datos con pandas (más robusto para manejar codificaciones)
        print("Leyendo datos de SQLite...")
        df = pd.read_sql_query("SELECT * FROM actividades", sqlite_conn)
        print(f"Leídos {len(df)} registros")
        
        # Configuración de PostgreSQL - ASCII simple
        print("Configurando conexión a PostgreSQL...")
        pg_params = "host=localhost dbname=actividades_infantiles user=postgres password=password"
        
        # Conectar a PostgreSQL
        print("Conectando a PostgreSQL...")
        pg_conn = psycopg2.connect(pg_params)
        pg_cursor = pg_conn.cursor()
        
        # Crear tabla en PostgreSQL
        print("Creando tabla en PostgreSQL...")
        pg_cursor.execute('''
        DROP TABLE IF EXISTS actividades;
        CREATE TABLE actividades (
            id SERIAL PRIMARY KEY,
            titulo TEXT,
            tipo TEXT,
            lugar TEXT,
            distrito TEXT,
            fecha_inicio TEXT,
            fecha_fin TEXT,
            url TEXT
        );
        ''')
        
        # Insertar datos fila por fila
        print("Insertando datos en PostgreSQL...")
        for _, row in df.iterrows():
            # Reemplazar NaN con None
            row = row.where(pd.notnull(row), None)
            
            pg_cursor.execute('''
            INSERT INTO actividades (titulo, tipo, lugar, distrito, fecha_inicio, fecha_fin, url)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
            ''', (
                row['titulo'], 
                row['tipo'], 
                row['lugar'], 
                row['distrito'], 
                row['fecha_inicio'], 
                row['fecha_fin'], 
                row['url']
            ))
        
        # Confirmar cambios
        pg_conn.commit()
        
        # Verificar la migración
        pg_cursor.execute("SELECT COUNT(*) FROM actividades")
        count = pg_cursor.fetchone()[0]
        print(f"Migración completada. {count} registros insertados en PostgreSQL.")
        
        # Cerrar conexiones
        pg_cursor.close()
        pg_conn.close()
        sqlite_conn.close()
        
    except Exception as e:
        print(f"Error: {e}")
        print(f"Tipo de error: {type(e)}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    print("Iniciando migración de SQLite a PostgreSQL...")
    migrate_sqlite_to_postgres()
    print("Proceso finalizado.")
""")

